In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from datetime import datetime, timedelta
from pyproj import Geod
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
indices = pd.MultiIndex.from_frame(pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])).reorder_levels([1,0])
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
demand = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'demand.csv'), index_col=0)
locations = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), parse_dates=['Time'])
actions = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), parse_dates=['Time'])

In [6]:
demand.index = pd.to_datetime(demand.index, format='%Y-%m-%d %H:%M')
locations.index = pd.MultiIndex.from_frame(locations.loc[:,['Time', 'Vehicle_Number_Plate']])
actions = pd.MultiIndex.from_frame(actions.loc[:,['Time', 'Vehicle_Number_Plate']])
locations.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)

In [8]:
it = indices.to_list()

In [39]:
ind = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])[['Time', 'Vehicle_Number_Plate']]

In [42]:
ind[100:200].index[20]

120

In [9]:
%timeit it[1000]
%timeit indices[1000]

65.9 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
31.5 µs ± 4.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [24]:
torch.tensor(locations.iloc[1000].values)==torch.tensor(tuple(locations.values)[1000])

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True])

In [27]:
lloc=list(locations.values)
tloc=tuple(locations.values)
%timeit torch.tensor(locations.loc[indices[1000]].values)
%timeit torch.tensor(locations.loc[it[1000][0], it[1000][1]].values)
%timeit torch.tensor(locations.iloc[1000].values)
%timeit torch.tensor(locations.values[1000])
%timeit torch.tensor(lloc[1000])
%timeit torch.tensor(tloc[1000])

526 µs ± 44.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
298 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
171 µs ± 7.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.6 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.58 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.52 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
at = actions.to_list()
%timeit indices[1000][0] in actions
%timeit indices[1000][0] in at
%timeit it[1000][0] in at


66.9 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
433 µs ± 54.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
375 µs ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
dt = demand.to_dict('index')
%timeit torch.tensor(demand.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(dt[indices[1000][0]].values()))
%timeit torch.tensor(tuple(dt[it[1000][0]].values()))

213 µs ± 9.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
46 µs ± 3.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.62 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
